In [1]:
import pandas as pd


In [2]:
from google.colab import files
uploaded = files.upload()

Saving job_postings.csv to job_postings.csv


In [3]:
df = pd.read_csv('job_postings.csv')

In [11]:
df.head()

,Job Posting ID,Job Posting Date,Job Title,Job Title Full,Job Position Type,Job Position Level,Years of Experience,Job Skills,Job Location,Minimum Pay,Maximum Pay,Pay Rate,Number of Applicants,Company Name,Company Industry,Company Size
0,2701524240,2017-01-01,Software Engineer,Software Engineer,Full-time,Entry level,1,"database, javascript, agile, linux, server, no...",United States,NaN,NaN,NaN,6.0,"Cardinal Financial Company, Limited Partnership",Financial Services,"1,001-5,000 employees"
1,2719108338,2017-01-01,Data Engineer,Senior Azure Data Engineer,Full-time,Mid-Senior level,2,"data_lake, cloud, python, spark, github, wareh...",United States,NaN,NaN,NaN,1.0,Brinks Home,Consumer Electronics,"1,001-5,000 employees"
2,2719503370,2017-01-01,Software Engineer,Software Engineer I,Full-time,Entry level,5,"mongo, oracle, microsoft, css, javascript, htm...",United States,NaN,NaN,NaN,16.0,Paycor,Computer Software,"1,001-5,000 employees"
3,2734877741,2017-01-01,Business Analyst,Associate Business Analyst,Full-time,Entry level,2,"agile, excel","Phoenix, AZ",NaN,NaN,NaN,1.0,Optum,Hospital & Health Care,"10,001+ employees"
4,2752415616,2017-01-01,Developer,Swift Developer,Contract,Mid-Senior level,4,"excel, back-end, ios, swift, programming","Richmond, CA",NaN,NaN,NaN,NaN,Toptal,Internet,"1,001-5,000 employees"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25114 entries, 0 to 25113
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Job Posting ID             25114 non-null  int64  
 1   Job Posting Date           25114 non-null  object 
 2   Job Title                  25114 non-null  object 
 3   Job Title Full             25114 non-null  object 
 4   Job Title Additional Info  8187 non-null   object 
 5   Job Position Type          25114 non-null  object 
 6   Job Position Level         25114 non-null  object 
 7   Years of Experience        25114 non-null  int64  
 8   Job Skills                 22904 non-null  object 
 9   Job Location               25114 non-null  object 
 10  Minimum Pay                1812 non-null   float64
 11  Maximum Pay                1812 non-null   float64
 12  Pay Rate                   1812 non-null   object 
 13  Number of Applicants       17529 non-null  flo

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np

In [7]:
from collections import Counter

In [8]:
all_skills = df['Job Skills'].dropna().str.split(', ').sum()
most_common_skills = [skill for skill, _ in Counter(all_skills).most_common(5)]


In [9]:
def generate_skills(row):
    if pd.isna(row['Job Skills']):
        if 'Data Scientist' in row['Job Title']:
            return 'python, machine learning, statistics, SQL'
        elif 'Data Analyst' in row['Job Title']:
            return 'excel, SQL, data visualization, statistics'
        elif 'Software Engineer' in row['Job Title']:
            return 'java, C++, python, agile'
        elif 'Data Engineer' in row['Job Title']:
            return 'ETL, data pipeline, python, cloud'
        elif 'Business Analyst' in row['Job Title']:
            return 'communication, requirements, SQL, documentation'
        else:
            return ', '.join(most_common_skills)  # Use most common skills
    return row['Job Skills']

In [12]:
df['Job Skills'] = df.apply(generate_skills, axis=1)

In [10]:
df = df.drop("Job Title Additional Info", axis=1)

In [13]:
median_pay = df.groupby(['Job Title', 'Job Position Type', 'Job Position Level'])[['Minimum Pay', 'Maximum Pay']].median()

In [14]:
df1 = df.copy()

In [15]:
def fill_pay(row):
    if pd.isna(row['Minimum Pay']) or pd.isna(row['Maximum Pay']):
        group_key = (row['Job Title'], row['Job Position Type'], row['Job Position Level'])
        if group_key in median_pay.index:
            if pd.isna(row['Minimum Pay']):
                row['Minimum Pay'] = median_pay.loc[group_key, 'Minimum Pay']
            if pd.isna(row['Maximum Pay']):
                row['Maximum Pay'] = median_pay.loc[group_key, 'Maximum Pay']
    return row

In [16]:
df = df.apply(fill_pay, axis=1)

In [17]:
df.describe()

,Job Posting ID,Years of Experience,Minimum Pay,Maximum Pay,Number of Applicants
count,2.511400e+04,25114.000000,24063.000000,24063.000000,17529.000000
mean,2.688359e+09,4.337700,99119.914391,135070.627104,14.833932
std,6.212277e+07,2.235979,26990.805545,29422.877624,23.457571
min,1.573681e+09,0.000000,28000.000000,42000.000000,1.000000
25%,2.678769e+09,3.000000,75000.000000,113000.000000,2.000000
50%,2.696126e+09,4.000000,106000.000000,143000.000000,6.000000
75%,2.721763e+09,6.000000,116000.000000,161000.000000,17.000000
max,2.797780e+09,25.000000,308000.000000,367000.000000,200.000000


In [18]:
from sklearn.ensemble import RandomForestRegressor

In [20]:
def fill_missing_pay_with_model(data, target_column):
    # Разделение на строки с и без пропусков
    train_data = data[data[target_column].notna()]
    test_data = data[data[target_column].isna()]

    # Если пропусков нет, возвращаем оригинальный датасет
    if test_data.empty:
        return data

    # Целевая переменная и признаки
    X_train = train_data.drop(columns=[target_column])
    y_train = train_data[target_column]
    X_test = test_data.drop(columns=[target_column])

    # Преобразование категориальных данных в числовые
    X_train = pd.get_dummies(X_train, drop_first=True)
    X_test = pd.get_dummies(X_test, drop_first=True)

    # Убедиться, что X_test и X_train имеют одинаковые колонки
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    # Обучение модели
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)

    # Предсказание пропущенных значений
    predicted_values = model.predict(X_test)

    # Заполнение пропусков
    data.loc[data[target_column].isna(), target_column] = predicted_values
    return data

# Заполнение пропусков в столбцах "Minimum Pay" и "Maximum Pay"
df1 = fill_missing_pay_with_model(df1, 'Minimum Pay')
df1 = fill_missing_pay_with_model(df1, 'Maximum Pay')

In [21]:
df1.describe()

,Job Posting ID,Years of Experience,Minimum Pay,Maximum Pay,Number of Applicants
count,2.511400e+04,25114.000000,25114.000000,25114.000000,17529.000000
mean,2.688359e+09,4.337700,92113.774389,126873.875926,14.833932
std,6.212277e+07,2.235979,13285.923116,16900.633553,23.457571
min,1.573681e+09,0.000000,28000.000000,42000.000000,1.000000
25%,2.678769e+09,3.000000,84160.000000,117760.000000,2.000000
50%,2.696126e+09,4.000000,91000.000000,125970.000000,6.000000
75%,2.721763e+09,6.000000,98170.000000,134360.000000,17.000000
max,2.797780e+09,25.000000,308000.000000,367000.000000,200.000000


In [22]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25114 entries, 0 to 25113
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Job Posting ID        25114 non-null  int64  
 1   Job Posting Date      25114 non-null  object 
 2   Job Title             25114 non-null  object 
 3   Job Title Full        25114 non-null  object 
 4   Job Position Type     25114 non-null  object 
 5   Job Position Level    25114 non-null  object 
 6   Years of Experience   25114 non-null  int64  
 7   Job Skills            25114 non-null  object 
 8   Job Location          25114 non-null  object 
 9   Minimum Pay           25114 non-null  float64
 10  Maximum Pay           25114 non-null  float64
 11  Pay Rate              1812 non-null   object 
 12  Number of Applicants  17529 non-null  float64
 13  Company Name          25053 non-null  object 
 14  Company Industry      24895 non-null  object 
 15  Company Size       

In [23]:
df1['Pay Rate'].unique()

array([nan, 'hr', 'yr'], dtype=object)

In [25]:
df_horly = df1[df1['Pay Rate'] == 'hr']
df_horly

,Job Posting ID,Job Posting Date,Job Title,Job Title Full,Job Position Type,Job Position Level,Years of Experience,Job Skills,Job Location,Minimum Pay,Maximum Pay,Pay Rate,Number of Applicants,Company Name,Company Industry,Company Size
9,2681258907,2017-01-02,Business Intelligence Analyst,Jr. Business Intelligence Analyst,Contract,Mid-Senior level,4,"r, scala, powershell, c++, java, python, table...","Raleigh, NC",71000.0,80000.0,hr,100.0,OnDemand Agility Solutions,Information Technology & Services,"1,001-5,000 employees"
26,2723217037,2017-01-04,Developer,Liferay Developer,Full-time,Mid-Senior level,6,"scrum, java, aws, agile","Atlanta, GA",162000.0,149000.0,hr,3.0,"ASAP Solutions Group, LLC",Management Consulting,"501-1,000 employees"
212,2748686994,2017-01-25,Data Analyst,Data Analyst,Contract,Associate,4,"powerpoint, coding, sql, powerbi",Dallas-Fort Worth Metroplex,68000.0,71000.0,hr,46.0,Apex Systems,Information Technology & Services,"1,001-5,000 employees"
239,2682234270,2017-01-28,Data Engineer,Data Engineer,Contract,Mid-Senior level,8,"cloud, aws, docker, nosql, apache, go, azure","Creve Coeur, MO",123000.0,147000.0,hr,7.0,Revolution Technologies,Staffing & Recruiting,"1,001-5,000 employees"
305,2690951888,2017-02-06,Operations Analyst,Sap Business Operations Analyst,Contract,Mid-Senior level,5,"excel, sap",United States,63000.0,84000.0,hr,43.0,Insight Global,Staffing & Recruiting,"1,001-5,000 employees"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24940,2702477685,2021-12-23,Data Analyst,Reporting And Data Analyst,Contract,Mid-Senior level,6,"excel, t-sql, power_bi, agile, sql, ssrs","Portland, Oregon Metropolitan Area",49000.0,85000.0,hr,31.0,Insight Global,Staffing & Recruiting,"1,001-5,000 employees"
24973,2738743166,2021-12-25,Software Engineer,Virtual Software Engineer,Full-time,Associate,4,"github, agile, python, matlab, c","Michigan, United States",138000.0,148000.0,hr,2.0,blueStone Staffing,Staffing & Recruiting,51-200 employees
24977,2695785947,2021-12-25,Data Engineer,Power Bi/Data Engineer,Contract,Mid-Senior level,4,"microsoft, dax, java, css, javascript, power_b...",United States,84000.0,124000.0,hr,25.0,"Take2 Consulting, LLC",Management Consulting,201-500 employees
24985,2734860098,2021-12-25,Data Engineer,Cyber Security Data Engineer,Contract,Mid-Senior level,8,"excel, power_bi, aws, mysql, python, programmi...","Hillsboro, OR",152000.0,158000.0,hr,NaN,VanderHouwen,Staffing & Recruiting,201-500 employees


In [26]:
df1 = df1.drop("Pay Rate", axis=1)

In [36]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25114 entries, 0 to 25113
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Job Posting ID        25114 non-null  int64  
 1   Job Posting Date      25114 non-null  object 
 2   Job Title             25114 non-null  object 
 3   Job Title Full        25114 non-null  object 
 4   Job Position Type     25114 non-null  object 
 5   Job Position Level    25114 non-null  object 
 6   Years of Experience   25114 non-null  int64  
 7   Job Skills            25114 non-null  object 
 8   Job Location          25114 non-null  object 
 9   Minimum Pay           25114 non-null  float64
 10  Maximum Pay           25114 non-null  float64
 11  Number of Applicants  25114 non-null  float64
 12  Company Name          25114 non-null  object 
 13  Company Industry      25114 non-null  object 
 14  Company Size          25114 non-null  object 
dtypes: float64(3), int6

In [28]:
df2 = df1.copy()

In [30]:
df1['Company Industry'].fillna('Unknown', inplace=True)
df1['Company Size'].fillna('Unknown', inplace=True)

<ipython-input-30-21f90dd379fe>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['Company Industry'].fillna('Unknown', inplace=True)
<ipython-input-30-21f90dd379fe>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

In [32]:
df1['Company Name'].fillna('Unknown', inplace=True)

<ipython-input-32-7b382cd9c658>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['Company Name'].fillna('Unknown', inplace=True)


In [34]:
df1['Number of Applicants'].median()

6.0

In [35]:
df1['Number of Applicants'].fillna(df1['Number of Applicants'].median(), inplace=True)

<ipython-input-35-cc18d489f5bb>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['Number of Applicants'].fillna(df1['Number of Applicants'].median(), inplace=True)


In [37]:
df1.to_csv('cleaned_job.csv', index=False)